# Fermi photon fule and spacecraft data download

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
from time import sleep
import wget
import shutil
import os
import re

### references
# tutorial 1: https://medium.com/marketingdatascience/selenium%E6%95%99%E5%AD%B8-%E4%B8%80-%E5%A6%82%E4%BD%95%E4%BD%BF%E7%94%A8webdriver-send-keys-988816ce9bed
# tutorial 2: https://ithelp.ithome.com.tw/articles/10277622
# selenium Locating Elements: https://selenium-python.readthedocs.io/locating-elements.html
###

### read in necessary parameters
table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/data/BZCAT_Xmatch_ACC_list.csv')

table_name = table.name.values
table_RA = table.RA.values
table_DEC = table.DEC.values

for i in range(93, len(table_name)):  ### len(table_name)
    
    test = 0
               
    ### test paramenerts setting
    # source_name = 'J0001-1551'  ### 'J0001-1551'  'J2158-3013'
    # input_coordinate = '0.272042 -15.851939'
    # radius = 1
    # date = ""  # set " "" " for default ### 2017-12-25 00:00:00, 2022-12-26 00:00:00
    
    ### for loop parameters setting
    source_name = table_name[i]
    input_coordinate = str(table_RA[i] ) +' '+ str(table_DEC[i] ) 
    radius = 1
    date = ""  # set " "" " for default => last six months of data will be download
    
    path = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/Fermi_all_sky_survey_data/'+source_name

    print('i =', i, ', source name:', source_name)

    ### create folder for data
    if not os.path.exists(path):
        os.mkdir(path)
    
    #######################################################################################################
    ### start web crawler
    #######################################################################################################
    try:
        #######################################################################################################
        ### page 1  
        #######################################################################################################
        driver = webdriver.Chrome()

        ### location of data download
        driver.get("https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi") 

        ### coordinate
        coor = driver.find_element(By.NAME, "coordfield")
        coor.send_keys(input_coordinate)

        ### search radius
        search_rad = driver.find_element(By.NAME, "shapefield")
        search_rad.send_keys(radius)

        ### search date
        search_date = driver.find_element(By.NAME, "timefield")
        search_date.send_keys(date)

        ### proceed search
        search = driver.find_element(By.XPATH, "//input[@value = 'Start Search']")
        search.click()
        
        #######################################################################################################
        ### page 2
        #######################################################################################################
        queue_result = driver.find_element(By.ID, "sec-wrapper")
        text = queue_result.text

        ### get website estimated proceeding time    
        initial = 0
        content = []
        for k in range(0, len(text)): 
            if text[k:k+1] == '\n':
                content.append(text[initial:k])
                initial = k+1
            elif k == len(text)-1:
                content.append(text[initial:k+1])
        countdown = content[-1]
        time = re.findall(r'\d+', countdown)[0] ### find the first number
        
        print('wating for', float(time), 'seconds...')
        sleep(float(time))
        
        ### search id
        id_info = driver.find_element(By.ID, 'sec-wrapper')

        ### get id information
        id_name = id_info.text[6:29]

        link = 'https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/QueryResults.cgi?id='+id_name
        driver.get(link) 
        
        #######################################################################################################
        ### page 3
        #######################################################################################################
#         queue_result = driver.find_element(By.CLASS_NAME, "lat-query")
#         text = queue_result.text

#         sleep(2)

#         ### refresh website
#         driver.refresh()

#         ### get website estimated proceeding time    
#         initial = 0
#         content = []
#         for k in range(0, len(text)): 
#             if text[k:k+1] == '\n':
#                 content.append(text[initial:k])
#                 initial = k+1
#             elif k == len(text)-1:
#                 content.append(text[initial:k+1])

#         ### get waiting time 
#         time = np.ceil(float( content[-1] ))  # [82:89]

#         ### wait for proceeding
#         print("waiting for", time+5, "seconds...")
#         sleep(time+5)

#         ### refresh website
#         driver.refresh()

        ### find data download link
        result = driver.find_element(By.ID, "sec-wrapper")

        text = result.text

        ### classified sentence into deiifernt sub-array
        initial = 0
        content = []
        for k in range(0, len(text)): 
            if text[k:k+1] == '\n':
                content.append(text[initial:k])
                initial = k+1
            elif k == len(text)-1:
                content.append(text[initial:k+1])

        array = []
        for k in range(0, len(content)):
            if np.where( content[k][0:4] == 'wget' )[0] == 0 :
                array.append(k)

        ### get data download url
        url_array = np.array(content)[ array ]

        ### data download
        for k in range(0, len(url_array)):
            url = url_array[k][5::]
            print('file name =', url[-9:-1])
            ### download data using "wget"
            wget.download(url)

            print('\n')
            ### move downloaded data into a new folder
            file_name = url[-33::] 
            shutil.move("/Users/87steven/Documents/ASIAA/Blazar SED code and data/"+file_name, path+"/"+url[-9:-1]+"s")

            sleep(1)

        driver.close() # 關閉瀏覽器視窗 (use after data download has been proceed)
    except Exception as error:
        test = 1
        print('i = ', i ,', Error occured:', error)
        print('-')
    
    while test == 1:
        print('retring...')
        print('i =', i, ', source name:', source_name)
        print('-')
        
        try:
            #######################################################################################################
            ### page 1  
            #######################################################################################################
            driver = webdriver.Chrome()

            ### location of data download
            driver.get("https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi") 

            ### coordinate
            coor = driver.find_element(By.NAME, "coordfield")
            coor.send_keys(input_coordinate)

            ### search radius
            search_rad = driver.find_element(By.NAME, "shapefield")
            search_rad.send_keys(radius)

            ### search date
            search_date = driver.find_element(By.NAME, "timefield")
            search_date.send_keys(date)

            ### proceed search
            search = driver.find_element(By.XPATH, "//input[@value = 'Start Search']")
            search.click()
            
            #######################################################################################################
            ### page 2  
            #######################################################################################################
            queue_result = driver.find_element(By.ID, "sec-wrapper")
            text = queue_result.text

            ### get website estimated proceeding time (the page after search, page 2)
            initial = 0
            content = []
            for k in range(0, len(text)): 
                if text[k:k+1] == '\n':
                    content.append(text[initial:k])
                    initial = k+1
                elif k == len(text)-1:
                    content.append(text[initial:k+1])
                    
            countdown = content[-1]
            time = re.findall(r'\d+', countdown)[0] ### find the first number

            print('wating for', float(time), 'seconds...')
            sleep(float(time))

            ### search id
            id_info = driver.find_element(By.ID, 'sec-wrapper')

            ### get id information
            id_name = id_info.text[6:29]

            link = 'https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/QueryResults.cgi?id='+id_name
            driver.get(link) 
            
            #######################################################################################################
            ### page 3
            #######################################################################################################
#             queue_result = driver.find_element(By.CLASS_NAME, "lat-query")
#             text = queue_result.text

#             sleep(2)

#             ### refresh website
#             driver.refresh()

#             ### get website estimated proceeding time    
#             initial = 0
#             content = []
#             for k in range(0, len(text)): 
#                 if text[k:k+1] == '\n':
#                     content.append(text[initial:k])
#                     initial = k+1
#                 elif k == len(text)-1:
#                     content.append(text[initial:k+1])

#             ### get waiting time 
#             time = np.ceil(float( content[-1] ))  # [82:89]

#             ### wait for proceeding
#             print("waiting for", time+5, "seconds...")
#             sleep(time+5)

#             ### refresh website
#             driver.refresh()

            ### find data download link
            result = driver.find_element(By.ID, "sec-wrapper")

            text = result.text

            ### classified sentence into deiifernt sub-array
            initial = 0
            content = []
            for k in range(0, len(text)): 
                if text[k:k+1] == '\n':
                    content.append(text[initial:k])
                    initial = k+1
                elif k == len(text)-1:
                    content.append(text[initial:k+1])

            array = []
            for k in range(0, len(content)):
                if np.where( content[k][0:4] == 'wget' )[0] == 0 :
                    array.append(k)

            ### get data download url
            url_array = np.array(content)[ array ]

            ### data download
            for k in range(0, len(url_array)):
                url = url_array[k][5::]
                print('file name =', url[-9:-1])
                ### download data using "wget"
                wget.download(url)

                print('\n')
                ### move downloaded data into a new folder
                file_name = url[-33::] 
                shutil.move("/Users/87steven/Documents/ASIAA/Blazar SED code and data/"+file_name, path+"/"+url[-9:-1]+"s")

                sleep(1)

            driver.close() # 關閉瀏覽器視窗 (use after data download has been proceed)
            
            test = 0
        except Exception as error:
            test = 1
            print('Error occured:', error)
            print('-')

    print('=================================== File saved succfully ===================================')


In [ ]:
### check what file is empty
empty_folder_name = []

for i in range(0, len(table_name)): # len(table_name)
    path = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/Fermi_all_sky_survey_data/'+table_name[i]
    
    if os.path.exists(path):
        #print('True')
        if len( os.listdir(path) ) == 0: ### is the file is enpty
            empty_folder_name.append( table_name[i] )


In [ ]:
### check what file have not yet download (folder have not yet been create)
from os import listdir
from os.path import isfile, isdir, join

path = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/Fermi_all_sky_survey_data/'

files = listdir(path)

miss_name = np.setdiff1d( np.array(table_name), np.array(files) ) ### check what source is missing
